## Data prep for MR

#### Thank you Storm et al: https://www.nature.com/articles/s41467-021-26280-1 and Mishra et al: https://www.researchgate.net/publication/357596053_Stroke_genetics_informs_drug_discovery_and_risk_prediction_across_ancestries

In [1]:
import pandas as pd
import numpy as np

In [4]:
%%bash
dx download ALS_MR/eQTL_data/eqtl_data_eqtlgen/2019-12-11-cis-eQTLsFDR0.05-ProbeLevel-CohortInfoRemoved-BonferroniAdded.txt.gz

In [6]:
eqtlgen = pd.read_csv('2019-12-11-cis-eQTLsFDR0.05-ProbeLevel-CohortInfoRemoved-BonferroniAdded.txt.gz', delim_whitespace=True)
eqtlgen.head()

,Pvalue,SNP,SNPChr,SNPPos,AssessedAllele,OtherAllele,Zscore,Gene,GeneSymbol,GeneChr,GenePos,NrCohorts,NrSamples,FDR,BonferroniP
0,3.271700e-310,rs12230244,12,10117369,T,A,200.7534,ENSG00000172322,CLEC12A,12,10126104,34,30596,0.0,4.166200e-302
1,3.271700e-310,rs12229020,12,10117683,G,C,200.6568,ENSG00000172322,CLEC12A,12,10126104,34,30596,0.0,4.166200e-302
2,3.271700e-310,rs61913527,12,10116198,T,C,200.2654,ENSG00000172322,CLEC12A,12,10126104,34,30598,0.0,4.166200e-302
3,3.271700e-310,rs2594103,12,10115428,T,C,200.0420,ENSG00000172322,CLEC12A,12,10126104,34,30598,0.0,4.166200e-302
4,3.271700e-310,rs12231833,12,10118428,A,G,199.9508,ENSG00000172322,CLEC12A,12,10126104,34,30592,0.0,4.166200e-302


In [7]:
%%bash
dx download ALS_MR/eQTL_data/eqtl_data_eqtlgen/2018-07-18_SNP_AF_for_AlleleB_combined_allele_counts_and_MAF_pos_added.txt.gz

In [8]:
allele = pd.read_csv('2018-07-18_SNP_AF_for_AlleleB_combined_allele_counts_and_MAF_pos_added.txt.gz', delim_whitespace=True)
allele.head()

,SNP,hg19_chr,hg19_pos,AlleleA,AlleleB,allA_total,allAB_total,allB_total,AlleleB_all
0,rs140337953,1,30923,T,G,71.0,10.0,1.0,0.073171
1,rs146477069,1,54421,A,G,75.0,6.0,0.0,0.037037
2,rs141149254,1,54490,G,A,381.0,122.0,5.0,0.129921
3,rs2462492,1,54676,C,T,54.0,64.0,29.0,0.414966
4,rs143174675,1,54753,T,G,73.0,8.0,1.0,0.060976


In [9]:
%%bash
dx download ALS_MR/druggable_genome.csv

In [12]:
dg = pd.read_csv('druggable_genome.csv')
dg.head()

,ensembl_gene_id,druggability_tier,hgnc_names,chr_b37,start_b37,end_b37,strand,description,no_of_gwas_regions,small_mol_druggable,bio_druggable,adme_gene
0,ENSG00000000938,Tier 1,FGR,1,27938575,27961788,-1,feline Gardner-Rasheed sarcoma viral oncogene ...,0,Y,N,N
1,ENSG00000001626,Tier 1,CFTR,7,117105838,117356025,1,cystic fibrosis transmembrane conductance regu...,0,Y,N,Y
2,ENSG00000001630,Tier 1,CYP51A1,7,91741465,91772266,-1,"cytochrome P450, family 51, subfamily A, polyp...",0,Y,N,Y
3,ENSG00000002549,Tier 1,LAP3,4,17578815,17609595,1,leucine aminopeptidase 3 [Source:HGNC Symbol;A...,0,Y,N,N
4,ENSG00000004468,Tier 1,CD38,4,15779898,15854853,1,CD38 molecule [Source:HGNC Symbol;Acc:1667],0,N,Y,N


In [13]:
dg_tier1 = dg[dg['druggability_tier']=='Tier 1']
dg_t1_nox = dg_tier1[dg_tier1['chr_b37'] != 'X']
dg_t1_nox.info()

<class 'pandas.core.frame.DataFrame'>

Int64Index: 1377 entries, 0 to 1426

Data columns (total 12 columns):

ensembl_gene_id        1377 non-null object

druggability_tier      1377 non-null object

hgnc_names             1375 non-null object

chr_b37                1377 non-null object

start_b37              1377 non-null int64

end_b37                1377 non-null int64

strand                 1377 non-null int64

description            1377 non-null object

no_of_gwas_regions     1377 non-null int64

small_mol_druggable    1377 non-null object

bio_druggable          1377 non-null object

adme_gene              1377 non-null object

dtypes: int64(4), object(8)

memory usage: 139.9+ KB


In [14]:
pos = dg_t1_nox[['ensembl_gene_id', 'start_b37', 'end_b37']]
pos.head()

,ensembl_gene_id,start_b37,end_b37
0,ENSG00000000938,27938575,27961788
1,ENSG00000001626,117105838,117356025
2,ENSG00000001630,91741465,91772266
3,ENSG00000002549,17578815,17609595
4,ENSG00000004468,15779898,15854853


In [16]:
gene_list = dg_t1_nox['ensembl_gene_id']
eqtl_filter= eqtlgen[eqtlgen['Gene'].isin(gene_list)]
eqtl_filter.info()

<class 'pandas.core.frame.DataFrame'>

Int64Index: 668141 entries, 13 to 10507655

Data columns (total 15 columns):

Pvalue            668141 non-null float64

SNP               668141 non-null object

SNPChr            668141 non-null int64

SNPPos            668141 non-null int64

AssessedAllele    668141 non-null object

OtherAllele       668141 non-null object

Zscore            668141 non-null float64

Gene              668141 non-null object

GeneSymbol        668141 non-null object

GeneChr           668141 non-null int64

GenePos           668141 non-null int64

NrCohorts         668141 non-null int64

NrSamples         668141 non-null int64

FDR               668141 non-null float64

BonferroniP       668141 non-null float64

dtypes: float64(4), int64(6), object(5)

memory usage: 81.6+ MB


In [17]:
merge1 = pd.merge(eqtl_filter, pos, left_on='Gene', right_on='ensembl_gene_id', how='outer')
merge1.head()

,Pvalue,SNP,SNPChr,SNPPos,AssessedAllele,OtherAllele,Zscore,Gene,GeneSymbol,GeneChr,GenePos,NrCohorts,NrSamples,FDR,BonferroniP,ensembl_gene_id,start_b37,end_b37
0,3.271700e-310,rs964611,15.0,48597514.0,A,C,193.7198,ENSG00000074803,SLC12A1,15.0,48540068.0,15.0,13755.0,0.0,4.166200e-302,ENSG00000074803,48483861,48596275
1,3.271700e-310,rs61248772,15.0,48598726.0,T,A,192.9769,ENSG00000074803,SLC12A1,15.0,48540068.0,15.0,13755.0,0.0,4.166200e-302,ENSG00000074803,48483861,48596275
2,3.271700e-310,rs74011998,15.0,48596713.0,C,T,192.9409,ENSG00000074803,SLC12A1,15.0,48540068.0,15.0,13753.0,0.0,4.166200e-302,ENSG00000074803,48483861,48596275
3,3.271700e-310,rs7168752,15.0,48600780.0,C,T,192.7138,ENSG00000074803,SLC12A1,15.0,48540068.0,15.0,13755.0,0.0,4.166200e-302,ENSG00000074803,48483861,48596275
4,3.271700e-310,rs79040993,15.0,48602146.0,T,G,192.4130,ENSG00000074803,SLC12A1,15.0,48540068.0,15.0,13754.0,0.0,4.166200e-302,ENSG00000074803,48483861,48596275


In [25]:
cols = merge1.columns
gene = merge1['GeneSymbol'].unique()

eqtl_5kb = pd.DataFrame(columns=cols)

merge1['SNPPos'] = pd.to_numeric(merge1['SNPPos'])
merge1['start_b37'] = pd.to_numeric(merge1['start_b37'])
merge1['end_b37'] = pd.to_numeric(merge1['end_b37'])


for i in gene:

    gene_fil = i
    dat1 = merge1[merge1['GeneSymbol']==gene_fil]
    dat1 = dat1[(dat1['SNPPos'] > dat1['start_b37']-5000) & (dat1['SNPPos'] < dat1['end_b37']+5000)]
    
    eqtl_5kb = eqtl_5kb.append(dat1)

In [28]:
eqtl_5kb.info()

<class 'pandas.core.frame.DataFrame'>

Int64Index: 96361 entries, 0 to 668137

Data columns (total 18 columns):

Pvalue             96361 non-null float64

SNP                96361 non-null object

SNPChr             96361 non-null float64

SNPPos             96361 non-null float64

AssessedAllele     96361 non-null object

OtherAllele        96361 non-null object

Zscore             96361 non-null float64

Gene               96361 non-null object

GeneSymbol         96361 non-null object

GeneChr            96361 non-null float64

GenePos            96361 non-null float64

NrCohorts          96361 non-null float64

NrSamples          96361 non-null float64

FDR                96361 non-null float64

BonferroniP        96361 non-null float64

ensembl_gene_id    96361 non-null object

start_b37          96361 non-null object

end_b37            96361 non-null object

dtypes: float64(10), object(8)

memory usage: 14.0+ MB


In [29]:
eqtl_5kb.head()

,Pvalue,SNP,SNPChr,SNPPos,AssessedAllele,OtherAllele,Zscore,Gene,GeneSymbol,GeneChr,GenePos,NrCohorts,NrSamples,FDR,BonferroniP,ensembl_gene_id,start_b37,end_b37
0,3.271700e-310,rs964611,15.0,48597514.0,A,C,193.7198,ENSG00000074803,SLC12A1,15.0,48540068.0,15.0,13755.0,0.0,4.166200e-302,ENSG00000074803,48483861,48596275
1,3.271700e-310,rs61248772,15.0,48598726.0,T,A,192.9769,ENSG00000074803,SLC12A1,15.0,48540068.0,15.0,13755.0,0.0,4.166200e-302,ENSG00000074803,48483861,48596275
2,3.271700e-310,rs74011998,15.0,48596713.0,C,T,192.9409,ENSG00000074803,SLC12A1,15.0,48540068.0,15.0,13753.0,0.0,4.166200e-302,ENSG00000074803,48483861,48596275
3,3.271700e-310,rs7168752,15.0,48600780.0,C,T,192.7138,ENSG00000074803,SLC12A1,15.0,48540068.0,15.0,13755.0,0.0,4.166200e-302,ENSG00000074803,48483861,48596275
11,3.271700e-310,rs7162936,15.0,48594608.0,C,G,159.4042,ENSG00000074803,SLC12A1,15.0,48540068.0,6.0,9671.0,0.0,4.166200e-302,ENSG00000074803,48483861,48596275


In [30]:
allele_info = allele[['SNP','AlleleB', 'AlleleB_all']]

In [31]:
merge2 = pd.merge(eqtl_5kb, allele_info, on='SNP' )
merge2.head()

,Pvalue,SNP,SNPChr,SNPPos,AssessedAllele,OtherAllele,Zscore,Gene,GeneSymbol,GeneChr,GenePos,NrCohorts,NrSamples,FDR,BonferroniP,ensembl_gene_id,start_b37,end_b37,AlleleB,AlleleB_all
0,3.271700e-310,rs964611,15.0,48597514.0,A,C,193.7198,ENSG00000074803,SLC12A1,15.0,48540068.0,15.0,13755.0,0.0,4.166200e-302,ENSG00000074803,48483861,48596275,A,0.156582
1,3.271700e-310,rs61248772,15.0,48598726.0,T,A,192.9769,ENSG00000074803,SLC12A1,15.0,48540068.0,15.0,13755.0,0.0,4.166200e-302,ENSG00000074803,48483861,48596275,T,0.156809
2,3.271700e-310,rs74011998,15.0,48596713.0,C,T,192.9409,ENSG00000074803,SLC12A1,15.0,48540068.0,15.0,13753.0,0.0,4.166200e-302,ENSG00000074803,48483861,48596275,C,0.156557
3,3.271700e-310,rs7168752,15.0,48600780.0,C,T,192.7138,ENSG00000074803,SLC12A1,15.0,48540068.0,15.0,13755.0,0.0,4.166200e-302,ENSG00000074803,48483861,48596275,C,0.156582
4,3.271700e-310,rs7162936,15.0,48594608.0,C,G,159.4042,ENSG00000074803,SLC12A1,15.0,48540068.0,6.0,9671.0,0.0,4.166200e-302,ENSG00000074803,48483861,48596275,C,0.156247


In [60]:
flip = merge2[merge2['AssessedAllele'] != merge2['AlleleB']]
flip.head()

,Pvalue,SNP,SNPChr,SNPPos,AssessedAllele,OtherAllele,Zscore,Gene,GeneSymbol,GeneChr,...,NrCohorts,NrSamples,FDR,BonferroniP,ensembl_gene_id,start_b37,end_b37,AlleleB,AlleleB_all,new_f
5,3.271700e-310,rs11855410,15.0,48593854.0,C,T,122.8678,ENSG00000074803,SLC12A1,15.0,...,15.0,13742.0,0.0,4.166200e-302,ENSG00000074803,48483861,48596275,T,0.817454,0.817454
10,3.271700e-310,rs72623972,15.0,48586994.0,C,T,115.8559,ENSG00000074803,SLC12A1,15.0,...,15.0,13755.0,0.0,4.166200e-302,ENSG00000074803,48483861,48596275,T,0.819253,0.819253
11,3.271700e-310,rs8026751,15.0,48582845.0,A,G,115.6258,ENSG00000074803,SLC12A1,15.0,...,15.0,13755.0,0.0,4.166200e-302,ENSG00000074803,48483861,48596275,G,0.819053,0.819053
12,3.271700e-310,rs7174204,15.0,48579146.0,G,C,115.1496,ENSG00000074803,SLC12A1,15.0,...,15.0,13755.0,0.0,4.166200e-302,ENSG00000074803,48483861,48596275,C,0.815279,0.815279
15,3.271700e-310,rs60820695,15.0,48584883.0,T,C,110.9321,ENSG00000074803,SLC12A1,15.0,...,15.0,13755.0,0.0,4.166200e-302,ENSG00000074803,48483861,48596275,C,0.852457,0.852457


In [61]:
flip['new_f'] = 1 - flip['AlleleB_all']
flip.head()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 

A value is trying to be set on a copy of a slice from a DataFrame.

Try using .loc[row_indexer,col_indexer] = value instead



See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  """Entry point for launching an IPython kernel.


,Pvalue,SNP,SNPChr,SNPPos,AssessedAllele,OtherAllele,Zscore,Gene,GeneSymbol,GeneChr,...,NrCohorts,NrSamples,FDR,BonferroniP,ensembl_gene_id,start_b37,end_b37,AlleleB,AlleleB_all,new_f
5,3.271700e-310,rs11855410,15.0,48593854.0,C,T,122.8678,ENSG00000074803,SLC12A1,15.0,...,15.0,13742.0,0.0,4.166200e-302,ENSG00000074803,48483861,48596275,T,0.817454,0.182546
10,3.271700e-310,rs72623972,15.0,48586994.0,C,T,115.8559,ENSG00000074803,SLC12A1,15.0,...,15.0,13755.0,0.0,4.166200e-302,ENSG00000074803,48483861,48596275,T,0.819253,0.180747
11,3.271700e-310,rs8026751,15.0,48582845.0,A,G,115.6258,ENSG00000074803,SLC12A1,15.0,...,15.0,13755.0,0.0,4.166200e-302,ENSG00000074803,48483861,48596275,G,0.819053,0.180947
12,3.271700e-310,rs7174204,15.0,48579146.0,G,C,115.1496,ENSG00000074803,SLC12A1,15.0,...,15.0,13755.0,0.0,4.166200e-302,ENSG00000074803,48483861,48596275,C,0.815279,0.184721
15,3.271700e-310,rs60820695,15.0,48584883.0,T,C,110.9321,ENSG00000074803,SLC12A1,15.0,...,15.0,13755.0,0.0,4.166200e-302,ENSG00000074803,48483861,48596275,C,0.852457,0.147543


In [66]:
mergetemp= merge2[~merge2['SNP'].isin(flip['SNP'])]
mergetemp['new_f'] = mergetemp['AlleleB_all']

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 

A value is trying to be set on a copy of a slice from a DataFrame.

Try using .loc[row_indexer,col_indexer] = value instead



See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  


In [69]:
mergefinal = mergetemp.append(flip)
mergefinal.tail()

,Pvalue,SNP,SNPChr,SNPPos,AssessedAllele,OtherAllele,Zscore,Gene,GeneSymbol,GeneChr,...,NrCohorts,NrSamples,FDR,BonferroniP,ensembl_gene_id,start_b37,end_b37,AlleleB,AlleleB_all,new_f
96340,0.000011,rs75702571,2.0,39487073.0,A,T,4.3949,ENSG00000011566,MAP4K3,2.0,...,14.0,8850.0,0.028673,1.0,ENSG00000011566,39476407,39664453,T,0.827448,0.172552
96348,0.000015,rs2717701,16.0,3066168.0,C,G,-4.3306,ENSG00000006327,TNFRSF12A,16.0,...,33.0,23965.0,0.037874,1.0,ENSG00000006327,3068446,3072384,G,0.627290,0.372710
96349,0.000016,rs2257295,16.0,3065596.0,C,T,-4.3189,ENSG00000006327,TNFRSF12A,16.0,...,32.0,23850.0,0.039721,1.0,ENSG00000006327,3068446,3072384,T,0.629747,0.370253
96350,0.000002,rs701929,1.0,211846876.0,G,A,4.7223,ENSG00000117650,NEK2,1.0,...,35.0,31352.0,0.006332,1.0,ENSG00000117650,211836114,211848960,A,0.498415,0.501585
96351,0.000002,rs697003,1.0,211843076.0,G,C,4.7198,ENSG00000117650,NEK2,1.0,...,35.0,31349.0,0.006385,1.0,ENSG00000117650,211836114,211848960,C,0.497773,0.502227


In [75]:
#Beta = z / sqrt(2p(1− p)(n + z^2)) and
#SE =1 / sqrt(2p(1− p)(n + z^2))

mergefinal['Zscore'] = pd.to_numeric(mergefinal['Zscore'])
mergefinal['new_f'] = pd.to_numeric(mergefinal['new_f'])
mergefinal['NrSamples'] = pd.to_numeric(mergefinal['NrSamples'])

mergefinal['beta'] = mergefinal['Zscore'] / np.sqrt(((2 * mergefinal['new_f']) * (1 - mergefinal['new_f'])) * (mergefinal['NrSamples'] + (mergefinal['Zscore']**2)))

mergefinal['se'] = 1 / np.sqrt((2 * mergefinal['new_f']) * (1 - mergefinal['new_f']) * (mergefinal['NrSamples'] + mergefinal['Zscore']**2))



In [76]:
mergefinal.head()

,Pvalue,SNP,SNPChr,SNPPos,AssessedAllele,OtherAllele,Zscore,Gene,GeneSymbol,GeneChr,...,FDR,BonferroniP,ensembl_gene_id,start_b37,end_b37,AlleleB,AlleleB_all,new_f,beta,se
0,3.271700e-310,rs964611,15.0,48597514.0,A,C,193.7198,ENSG00000074803,SLC12A1,15.0,...,0.0,4.166200e-302,ENSG00000074803,48483861,48596275,A,0.156582,0.156582,1.664494,0.008592
1,3.271700e-310,rs61248772,15.0,48598726.0,T,A,192.9769,ENSG00000074803,SLC12A1,15.0,...,0.0,4.166200e-302,ENSG00000074803,48483861,48596275,T,0.156809,0.156809,1.661796,0.008611
2,3.271700e-310,rs74011998,15.0,48596713.0,C,T,192.9409,ENSG00000074803,SLC12A1,15.0,...,0.0,4.166200e-302,ENSG00000074803,48483861,48596275,C,0.156557,0.156557,1.662835,0.008618
3,3.271700e-310,rs7168752,15.0,48600780.0,C,T,192.7138,ENSG00000074803,SLC12A1,15.0,...,0.0,4.166200e-302,ENSG00000074803,48483861,48596275,C,0.156582,0.156582,1.662162,0.008625
4,3.271700e-310,rs7162936,15.0,48594608.0,C,G,159.4042,ENSG00000074803,SLC12A1,15.0,...,0.0,4.166200e-302,ENSG00000074803,48483861,48596275,C,0.156247,0.156247,1.657436,0.010398


In [80]:
eqtl_5kb_format = mergefinal[['GeneSymbol', 'SNP', 'beta', 'se', 'new_f', 'AssessedAllele', 'OtherAllele', 'Pvalue', 'NrSamples']]
eqtl_5kb_format.head()

,GeneSymbol,SNP,beta,se,new_f,AssessedAllele,OtherAllele,Pvalue,NrSamples
0,SLC12A1,rs964611,1.664494,0.008592,0.156582,A,C,3.271700e-310,13755.0
1,SLC12A1,rs61248772,1.661796,0.008611,0.156809,T,A,3.271700e-310,13755.0
2,SLC12A1,rs74011998,1.662835,0.008618,0.156557,C,T,3.271700e-310,13753.0
3,SLC12A1,rs7168752,1.662162,0.008625,0.156582,C,T,3.271700e-310,13755.0
4,SLC12A1,rs7162936,1.657436,0.010398,0.156247,C,G,3.271700e-310,9671.0


In [81]:
eqtl_5kb_format.to_csv('eqtl_5kb_format.txt', index=False, sep='\t')